# Fake Data Creator
## Imports and Helper Functions

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from random import uniform 
from faker import Factory,Faker
from collections import defaultdict
from sklearn.preprocessing import normalize,MinMaxScaler
from pandas import to_datetime
from random import randint
%matplotlib inline
fake = Faker()

In [2]:
def fixer(x1,mu1,std1,mu2):
    '''
    Fixes column values to be more realistic.
    ''' 
    std2 = mu1/4.1
    return ((x1-mu1)/(std1)) * (std2) + mu2

In [3]:
def realizer(data,minmax,rounder):
    mm = MinMaxScaler((minmax[0],minmax[1]))
    if rounder != '':
        return np.round(mm.fit_transform(data.as_matrix().reshape(-1,1)) , 0)  
    else:
        return np.round(mm.fit_transform(data.as_matrix().reshape(-1,1)) , 2)   

## Dictionary of Faker Providers

In [4]:
date_past = to_datetime('2006-01-01 11:42:52')
date_present = to_datetime('2017-01-01 11:42:52')

fake_codes = {}

fake_codes['phone_number'] = lambda x: fake.phone_number()
fake_codes['name'] = lambda x: fake.name()
fake_codes['catch_phrase'] = lambda x: fake.catch_phrase()
fake_codes['city'] = lambda x: fake.city()
fake_codes['binary'] = lambda x: randint(0,1)
fake_codes['randint1to10'] = lambda x: randint(1,10)
fake_codes['country'] = lambda x: fake.country()
fake_codes['timestamp_this_year'] = lambda x: fake.date_time_this_year()
fake_codes['timestamp'] = lambda x: fake.date_time_between_dates(date_past,date_present)
fake_codes['address'] = lambda x:  fake.address()
fake_codes['lot'] = lambda x:  fake.bothify()
fake_codes['AM or PM'] = lambda x:  fake.am_pm()
fake_codes['browser_info'] = lambda x:  fake.user_agent()
fake_codes['company'] = lambda x:  fake.company()
fake_codes['cc_num'] = lambda x:  fake.credit_card_number()
fake_codes['cc_exp'] = lambda x:  fake.credit_card_expire()
fake_codes['cc_sec_code'] = lambda x:  fake.credit_card_security_code()
fake_codes['cc_provider'] = lambda x:  fake.credit_card_provider()
fake_codes['email'] = lambda x:  fake.email()
fake_codes['job'] = lambda x:  fake.job()
fake_codes['ipv4'] = lambda x:  fake.ipv4()
fake_codes['language'] = lambda x:  fake.language_code()

_______

# Fake Classification Tasks

For classification tasks. The purpose of this function is to return a fake data set that can be used for classification. 


You pass in a list in the form 

    [(('col_name','fakercode',(min,max,mu))]

______

In [15]:
def fake_data_classification(dataname='myfakedata',nsamples=1,
                             nclass=2,datacode=[],target_name="target",pct_pos=0.5,std=2):
    """
    INPUT: Takes in a string of the dataname, and a dictionary of column names along with
           what they represent using the Faker() library.
    OUTPUT: Prints the head of the dataframe and also saves it to a csv file.
    """
    print("Generating Classification Data...")
    # Create Data
    data = datasets.make_blobs(n_samples=nsamples, n_features=len(datacode), 
                           centers=nclass, cluster_std=std)

    # Convert to DataFrames with normalized numbers
    features = pd.DataFrame(normalize(data[0])).apply(lambda x: x+1)
    features = pd.DataFrame(data[0])
    
    # Target
    target =  pd.DataFrame(data[1])
    target.columns = [target_name]
    
    print("Running Faker Code")
    for colind,(col_name,code,minmax,rounder) in enumerate(datacode):
        
        # Set Data to be Numerically Realistic
        features[colind] = realizer(features[colind],minmax,rounder)
        # Rename the column as required
        features=features.rename(columns = {colind:col_name})
        
        # Check to see if there is fake data to be generated!
        if code != 'none':
            features[col_name] = features[col_name].apply(fake_codes[code])
        
    ratio = pct_pos*(2/0.5)

    print("Completed Faker Generation")
    print("Saving as "+dataname+".csv")
    final_data = pd.concat([features,target],axis=1)
    
    # Fix issues with 
    num_pos = len(final_data[final_data[target_name]==1])
    positives = final_data[final_data[target_name]==1].sample(np.round(num_pos*ratio))
    negatives = final_data[final_data[target_name]==0]
    
    final_data = pd.concat([positives,negatives])
    final_data.to_csv(dataname+".csv",index=False)
    final_pos = len(final_data[final_data[target_name]==1])
    final_neg = len(final_data[final_data[target_name]==0])
    print("The number of positive points are: {}".format(final_pos))
    print("The number of negative points are: {}".format(final_neg))
    print("Percent of positive points is: {}".format(final_pos/(final_neg+final_pos)))

    
    print("Example of DataFrame Created:")
    print("\n")
    print(final_data.head())

In [24]:
def multi_fake_data_classification(dataname='myfakedata',nsamples=1,
                             nclass=3,datacode=[],target_name="target",std=2):
    """
    INPUT: Takes in a string of the dataname, and a dictionary of column names along with
           what they represent using the Faker() library.
    OUTPUT: Prints the head of the dataframe and also saves it to a csv file.
    """
    print("Generating Classification Data...")
    # Create Data
    data = datasets.make_blobs(n_samples=nsamples, n_features=len(datacode), 
                           centers=nclass, cluster_std=std)

    # Convert to DataFrames with normalized numbers
    features = pd.DataFrame(normalize(data[0])).apply(lambda x: x+1)
    features = pd.DataFrame(data[0])
    
    # Target
    target =  pd.DataFrame(data[1])
    target.columns = [target_name]
    
    print("Running Faker Code")
    for colind,(col_name,code,minmax,rounder) in enumerate(datacode):
        
        # Set Data to be Numerically Realistic
        features[colind] = realizer(features[colind],minmax,rounder)
        # Rename the column as required
        features=features.rename(columns = {colind:col_name})
        
        # Check to see if there is fake data to be generated!
        if code != 'none':
            features[col_name] = features[col_name].apply(fake_codes[code])
        
    

    print("Completed Faker Generation")
    print("Saving as "+dataname+".csv")
    final_data = pd.concat([features,target],axis=1)
    
    
    print("Example of DataFrame Created:")
    print("\n")
    print(final_data.describe())

In [21]:
def multi_fake_data_classification(dataname='myfakedata',nsamples=1,nclass=3,datacode=[],target_name="target",std=2):
    

    data = datasets.make_blobs(n_samples=nsamples, n_features=len(datacode), centers=nclass, cluster_std=std)
    return data

## Data Sets for clustering

In [25]:
# Hacker Data Set
base = [
    ("Session_Connection_Time",'none',(1,60),'yes'),
    ("Bytes Transferred",'none',(10,2000),''),
    ("Kali_Trace_Used",'binary',(0,1),''),
    ("Servers_Corrupted",'none',(1,10),''),
    ("Pages_Corrupted",'none',(3,15),'yes'),
    ("Location","country",(0,1),''),
    ("WPM_Typing_Speed",'none',(40,75),'')
]
result = multi_fake_data_classification('hack_data',nsamples=500,nclass=3,datacode=base,
                                  target_name="hack", std=1)

Generating Classification Data...
Running Faker Code
Completed Faker Generation
Saving as hack_data.csv
Example of DataFrame Created:


       Session_Connection_Time  Bytes Transferred  Kali_Trace_Used  \
count               500.000000         500.000000       500.000000   
mean                 35.624000        1148.456020         0.490000   
std                  18.874828         577.357338         0.500401   
min                   1.000000          10.000000         0.000000   
25%                  12.000000         462.907500         0.000000   
50%                  46.000000        1397.335000         0.000000   
75%                  50.000000        1610.045000         1.000000   
max                  60.000000        2000.000000         1.000000   

       Servers_Corrupted  Pages_Corrupted  WPM_Typing_Speed        hack  
count         500.000000       500.000000         500.00000  500.000000  
mean            6.127640         9.542000          60.86342    0.998000  
std        

(array([[-1.1381536 ,  4.92688901, -5.85879158, ...,  1.54835818,
         -7.65706875,  8.14730731],
        [-3.61286444, -2.56854496, -5.56765664, ...,  7.49127328,
          5.58157365, -5.00669104],
        [-0.27161734, -9.7048933 , -2.5822295 , ...,  0.49970095,
          1.37105124,  2.75386952],
        ..., 
        [-1.26937462, -8.3583158 , -3.81481804, ..., -0.87315573,
          2.00820047,  2.11995745],
        [-2.07290113,  3.20194004, -4.15905688, ..., -0.61489482,
         -7.15047156,  7.37089153],
        [-1.34311186, -8.36161986, -2.98110887, ...,  0.75193482,
         -0.03669774,  5.38140152]]),
 array([1, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 0, 2, 2,
        0, 1, 1, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 0, 0, 2, 2, 0, 0, 2, 0, 2, 1,
        2, 0, 2, 1, 0, 2, 1, 0, 0, 1, 0, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 0, 0,
        1, 2, 1, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        2, 0, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 1, 0, 2, 1, 2, 1, 2,

In [19]:
pd.read_csv('hack_data.csv').describe()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,WPM_Typing_Speed,hack
count,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.00000
mean,30.008982,607.245269,0.511976,5.258503,10.838323,57.342395,0.50000
std,14.088201,286.335932,0.500607,2.301907,3.063526,13.411063,0.50075
min,1.000000,10.000000,0.000000,1.000000,6.000000,40.000000,0.00000
25%,18.000000,372.200000,0.000000,3.122500,8.000000,44.127500,0.00000
50%,31.000000,601.650000,1.000000,5.285000,10.500000,57.840000,0.50000
75%,42.000000,843.702500,1.000000,7.400000,14.000000,70.577500,1.00000
max,60.000000,1330.500000,1.000000,10.000000,15.000000,75.000000,1.00000


## Creating Classification Sets

### Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

Here are the fields and their definitions:
* Name : Name of the latest contact at Company
* Age: Customer Age
* Total_Purchase: Total Ads Purchased
* Account_Manager: Binary 0=No manager, 1= Account manager assigned
* Years: Totaly Years as a customer
* Num_sites: Number of websites that use the service.
* Onboard_date: Date that the name of the latest contact was onboarded
* Location: Client HQ Address
* Company: Name of Client Company

In [172]:
base = [
    ('Names','name',(0,1),''),
    ("Age",'none',(22,65),'yes'),
    ("Total_Purchase",'none',(100,20000),''),
    ("Account_Manager",'binary',(0,1),''),
    ("Years",'none',(1,10),''),
    ("Num_Sites",'none',(3,15),'yes'),
    ("Onboard_date",'timestamp',(0,1),''),
    ("Location","address",(0,1),''),
    ("Company","company",(0,1),'')
]
result = fake_data_classification('new_customers',nsamples=10,nclass=2,datacode=base,
                                  target_name="Churn",pct_pos=0.05,std=6.2)

Generating Classification Data...
Running Faker Code
Completed Faker Generation
Saving as new_customers.csv
The number of positive points are: 1
The number of negative points are: 5
Percent of positive points is: 0.16666666666666666
Example of DataFrame Created:


            Names   Age  Total_Purchase  Account_Manager  Years  Num_Sites  \
2   Andrew Mccall  37.0         9935.53                1   7.71        8.0   
1  Michele Wright  23.0         7526.94                1   9.28       15.0   
3    Jeremy Chang  65.0          100.00                1   1.00       15.0   
4  Megan Ferguson  32.0         6487.50                0   9.40       14.0   
5    Taylor Young  32.0        13147.71                1  10.00        8.0   

         Onboard_date                                           Location  \
2 2011-08-29 18:37:54  38612 Johnny Stravenue\nNataliebury, WI 15717-...   
1 2013-07-22 18:19:54  21083 Nicole Junction Suite 332\nYoungport, ME...   
3 2006-12-11 07:48:13    085 Austin Vi

/Users/marci/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2572: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  locs = rs.choice(axis_length, size=n, replace=replace, p=weights)


### Check with X,y split
*Don't edit this!*

In [134]:
df = pd.read_csv("customer_churn.csv")
y = df['Churn']
X = df[['Age',"Total_Purchase","Account_Manager",'Years',"Num_Sites"]]
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.89      0.98      0.93       243
          1       0.81      0.46      0.59        54

avg / total       0.88      0.88      0.87       297



## Dog Food

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!

Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Other: Filler chemical that is not a preservative

In [140]:
base = [
     ('A','randint1to10',(0,1),''),
    ('B','randint1to10',(0,1),''),
    ('C','none',(5,15),'yes'),
    ('D','randint1to10',(0,1),''),
]

result = fake_data_classification('dog_food2',nsamples=700,nclass=2,datacode=base,
                                  target_name="Spoiled",pct_pos=0.1,std=3.2)

Generating Classification Data...
Running Faker Code
Completed Faker Generation
Saving as dog_food.csv
The number of positive points are: 140
The number of negative points are: 350
Percent of positive points is: 0.2857142857142857
Example of DataFrame Created:


     A  B     C  D  Spoiled
108  4  2  12.0  3        1
355  5  6  12.0  7        1
24   6  2  13.0  6        1
598  4  2  12.0  1        1
506  4  2  12.0  3        1


/Users/marci/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2572: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  locs = rs.choice(axis_length, size=n, replace=replace, p=weights)


In [146]:
df = pd.read_csv("dog_food.csv")
y = df['Spoiled']
X = df[['A',"B","C",'D']]
print(classification_report(y_test,predictions))
print(df.columns)
print(rfc.feature_importances_)

             precision    recall  f1-score   support

          0       0.99      1.00      1.00       114
          1       1.00      0.98      0.99        48

avg / total       0.99      0.99      0.99       162

Index(['A', 'B', 'C', 'D', 'Spoiled'], dtype='object')
[ 0.04022167  0.05269918  0.8641571   0.04292205]


### Test Validity with RF

In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [143]:
# X = df.drop('Class'.as_matrix()
# y = np.ravel(target.as_matrix())
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
rfc = RandomForestClassifier(200)
rfc.fit(X_train,y_train)   
predictions = rfc.predict(X_test)
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.99      1.00      1.00       114
          1       1.00      0.98      0.99        48

avg / total       0.99      0.99      0.99       162



## Cancer Detection

Just use breast cancer data

## Possible regression task with Facebook Data!

In [149]:
df = pd.read_csv('DataSets/dataset_Facebook.csv',sep=';')

_______

______

# Regression Tasks

In [70]:
def fake_data_regression(dataname='myfakedata',nsamples=1,datacode=[],target_name="target"):
    """
    INPUT: Takes in a string of the dataname, and a dictionary of column names along with
           what they represent using the Faker() library.
    OUTPUT: Prints the head of the dataframe and also saves it to a csv file.
    """
    print("Generating Classification Data...")
    std = uniform(1,2)
    print("Random std value was: "+str(std))
    # Create Data
    data = datasets.make_regression(n_samples=nsamples, n_features=len(datacode), 
                                    n_informative=len(datacode), n_targets=1,noise=0.2)
    
    # Convert to DataFrames with normalized numbers
    features = pd.DataFrame(normalize(data[0])).apply(lambda x: x+1)
    features = pd.DataFrame(data[0])
    
    # Target
    target =  pd.DataFrame(data[1])
    target.columns = [target_name]
    
    print("Running Faker Code")
    for colind,(col_name,code,minmax) in enumerate(datacode):
        
        # Set Data to be Numerically Realistic
        features[colind] = realizer(features[colind],minmax)
        # Rename the column as required
        features=features.rename(columns = {colind:col_name})
        
        # Check to see if there is fake data to be generated!
        if code != 'none':
            features[col_name] = features[col_name].apply(fake_codes[code])
        
        
    print("Completed Faker Generation")
    print("Saving as "+dataname+".csv")
    final_data = pd.concat([features,target],axis=1)
    final_data.to_csv(dataname+".csv")
    print("Example of DataFrame Created:")
    print("\n")
    print(final_data.head())
    return final_data
                           

In [ ]:
fake_data_regression()

_______